# Importing Libraries

In [1]:
from diff_weighted_fields import Grid1D, GaussianFieldGenerator1D, Zeldovich1D
import jax.numpy as jnp
import matplotlib.pyplot as plt
from jax.random import PRNGKey, split
import jax
from tqdm import tqdm

#grid setup
L = 4000
N = 1024
dk     = 10  #units of kf 
kmin   = 2  #units of kf
R_clip = 2  #units of cell size
kmax   =0.5 #units of kNyq

#define the field and gaussian generator
grid = Grid1D((N,), L, R_clip = R_clip, dk =dk, kmax = kmax , kmin =kmin)
mesh = Grid1D((N//2,), L, R_clip = R_clip, dk =dk, kmax = kmax , kmin =kmin)

def PK(k,theta, threshold = 0.001):
    A,R,n = theta
    _p = A*(k*R)**n*jnp.exp(-(k*R)**2)
    sig = jnp.max(_p)
    return _p + sig* threshold

Using clipping smoothing with R_clip = 2 cells (physical = 7.8125); k_clip = 8.042e-01
kmin: 0.0031415926535897933
kmax: 0.4021238684654236
dk: 0.015707963267948967
Using clipping smoothing with R_clip = 2 cells (physical = 15.625); k_clip = 4.021e-01
kmin: 0.0031415926535897933
kmax: 0.2010619342327118
dk: 0.015707963267948967


In [2]:
#white noise used to generate the field
keys = split(PRNGKey(2),300)
noise = jnp.array([grid.generate_hermitian_noise(x) for x in keys])

# Defining main functions 

In [3]:
#gaussian and zeldovich field generators
gen = GaussianFieldGenerator1D(grid, PK)
zel = Zeldovich1D(gen, mesh, scheme = 'cic')

In [4]:
Nx = 40000
Ny = 1

x_min, x_max = -1, 1
y_min, y_max = -0, 0

x_vec = jnp.linspace(x_min, x_max, Nx)
y_vec = jnp.linspace(y_min, y_max, Ny)

# --- (3) build a meshgrid and flatten it ---
# X, Y each have shape (Ny, Nx)
X, Y = jnp.meshgrid(x_vec, y_vec, indexing='xy')

# Now flatten:
#   xs_flat, ys_flat each shape (Ny*Nx,)
xs_flat = X.ravel()
ys_flat = Y.ravel()

# Build a 2D array of C‐vectors, shape (Ny*Nx, 4):
# each row = [ 1, x_i, y_i, 0 ]
ones_flat = jnp.ones_like(xs_flat)
zeros_flat = jnp.zeros_like(xs_flat)
C_flat = jnp.stack([ones_flat/5, xs_flat, ys_flat, zeros_flat], axis=1)  # shape (Ny*Nx, 4)

In [5]:
#fid params
A = 2.
R = 4*grid.H[0]
n = 2.
theta = jnp.array([A, R, n])
D = 1 #growth factor

#We define a batch of C vectors
BATCH_SIZE1 = 100
BATCH_SIZE_2 = 100
STEPS1 = C_flat.shape[0] // BATCH_SIZE1
STEPS2 = C_flat.shape[0] // BATCH_SIZE_2

In [6]:
def compute_pk(theta,D,C):
        pk_batch = zel.make_realization_batch(D,theta,0,C,noise)
        return jnp.mean(pk_batch, axis=1)

jac_pk = jax.jit(jax.jacfwd(compute_pk, argnums=0))

def compute_inv_cov(x):
    x = x - jnp.mean(x, axis=0)
    return jnp.linalg.inv((x.T @ x) / (x.shape[0] - 1))
compute_inv_cov_batch = jax.jit(jax.vmap(compute_inv_cov, in_axes=0))

In [9]:
%%timeit -n 2 -r 5 
pk_batch = zel.make_realization_batch(D,theta,0,C_flat[BATCH_SIZE1:2*BATCH_SIZE1],noise).block_until_ready()
cov_inv_batch = compute_inv_cov_batch(pk_batch).block_until_ready()

308 ms ± 40.9 ms per loop (mean ± std. dev. of 5 runs, 2 loops each)


In [10]:
jac_pk = jax.jit(jax.jacfwd(zel.make_realization_batch, argnums=1))

In [11]:
jac_pk(D,theta,0,C_flat[BATCH_SIZE1:2*BATCH_SIZE1],noise).block_until_ready()

Array([[[[ 2.7649415 ,  0.104816  , -2.7554743 ],
         [ 7.3373675 ,  0.6845547 , -7.7579837 ],
         [10.865695  ,  0.6515458 , -7.1037145 ],
         ...,
         [ 2.0239322 , -0.42556325,  1.0358725 ],
         [ 2.8282306 , -0.74098307,  2.100243  ],
         [ 1.5699457 , -0.49315172,  0.80276716]],

        [[ 1.0062507 , -0.07621747, -0.5796076 ],
         [ 4.5661592 ,  0.55822384, -5.563566  ],
         [ 6.8817434 ,  0.47083998, -3.9335353 ],
         ...,
         [ 0.85397214, -0.19344933,  0.06555378],
         [ 1.5222061 , -0.48382765,  0.7362293 ],
         [ 1.1925665 , -0.24281868,  0.135981  ]],

        [[ 2.7838984 ,  0.24103296, -3.8009446 ],
         [ 4.563619  ,  0.5102182 , -5.4828544 ],
         [15.692852  ,  1.088438  , -9.9684105 ],
         ...,
         [ 3.5279949 , -0.79876214,  1.0283215 ],
         [ 2.3781457 , -0.31409025, -0.30204022],
         [ 1.0202084 , -0.27161652,  0.13749008]],

        ...,

        [[ 1.6947043 ,  0.08264049, -2

In [12]:
%timeit -n 2 -r 5 jac_pk(D,theta,0,C_flat[2*BATCH_SIZE1:3*BATCH_SIZE1],noise).block_until_ready()

724 ms ± 72.6 ms per loop (mean ± std. dev. of 5 runs, 2 loops each)


In [ ]:
%timeit -n 2 -r 5 jac_pk(D,theta,0,C_flat[2*BATCH_SIZE1:3*BATCH_SIZE1],noise).block_until_ready()

In [11]:
cov_batch = compute_inv_cov_batch(pk_batch).block_until_ready()

In [12]:
J_batch = jac_pk(theta, D, C_flat[0:BATCH_SIZE]).block_until_ready()

In [ ]:
jac_pk(theta, D, C_flat[2*BATCH_SIZE:3*BATCH_SIZE]).block_until_ready()

In [9]:
pk_batch = zel.make_realization_batch(D,theta,0,C_flat[0:BATCH_SIZE],noise)
cov_batch = compute_inv_cov_batch(pk_batch)
J_batch = jac_pk(theta, D, C_flat[0:BATCH_SIZE])

In [14]:
pk_batch = zel.make_realization_batch(D,theta,0,C_flat[BATCH_SIZE:2*BATCH_SIZE],noise).block_until_ready()
cov_batch = compute_inv_cov_batch(pk_batch).block_until_ready()
J_batch = jac_pk(theta, D, C_flat[BATCH_SIZE:2*BATCH_SIZE]).block_until_ready()

In [ ]:
F = compute_batch(theta,D,C_flat[0:BATCH_SIZE])
F = F.block_until_ready()

In [ ]:
Fs = jnp.zeros(C_flat.shape[0])
for i in tqdm(range(STEPS)):
    start = i * BATCH_SIZE
    end = start + BATCH_SIZE
    _F = results_batch(theta, D, C_flat[start:end])
    Fs = Fs.at[start:end].set(_F)

In [ ]:
plt.plot(Fs)

In [13]:
def improved_compute_all_dets(theta, D, C_flat, batch_size, compute_batch_fn):
    """
    Compute determinants for all of C_flat by scanning over fixed‐size batches.

    Args:
      theta:      array of shape (3,)   (fiducial [A,R,n])
      D:          scalar               (growth factor)
      C_flat:     array (N_total, 4)   (all C rows stacked)
      batch_size: int (size of each minibatch)
      compute_batch_fn:  jit-compiled function
         signature: (theta, D, C_batch) -> shape (batch_size,)
         where C_batch has shape (batch_size, 4).

    Returns:
      dets_all:   array (N_total,) containing det F for each row of C_flat.
    """

    # 1) Figure out how many full batches we have:
    N_total = C_flat.shape[0]
    num_batches = N_total // batch_size

    # 2) Reshape C_flat into (num_batches, batch_size, 4)
    C_batches = C_flat.reshape((num_batches, batch_size, 4))

    # 3) Define the scan body.  We don’t need a “carry” here, so carry is just None.
    def scan_body(carry, C_batch):
        # compute_batch_fn returns a (batch_size,) array of det F for this batch
        dets = compute_batch_fn(theta, D, C_batch)
        return carry, dets

    # 4) Run lax.scan over all batches
    #    - carry_out is unused (None)
    #    - dets_stacked has shape (num_batches, batch_size)
    _, dets_stacked = jax.lax.scan(scan_body, None, C_batches)

    # 5) Flatten back to (N_total,)
    dets_all = dets_stacked.reshape((N_total,))
    return dets_all

In [14]:
improved_compute_all_dets_jit = jax.jit(
    improved_compute_all_dets,
    static_argnums=(3, 4)  
)

# Finally, call it:
#   Fs_all will have shape (N_total,)
Fs_all = improved_compute_all_dets_jit(theta, D, C_flat, BATCH_SIZE, results_batch)

In [ ]:
Fs_all

In [16]:
Fs_all = improved_compute_all_dets_jit(theta, D, C_flat, BATCH_SIZE, results_batch).block_until_ready()

In [17]:
Fs_all = improved_compute_all_dets(theta, D, C_flat, BATCH_SIZE, results_batch).block_until_ready() 